In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
#covert the time string to a number
def time_to_num(t):
    num = int(t[0:2]) * 3600 + int(t[3:5]) * 60 + int(t[6:8])
    return num

In [3]:
#generate the label
def get_label(df, future_step):
    label = ((df['AskPrice1'].shift(-future_step) + df['BidPrice1'].shift(-future_step)) 
    - (df['AskPrice1'] + df['BidPrice1'])) / 2
    return label

In [4]:
raw_data = pd.read_csv('data/data.csv')

In [5]:
raw_data['UpdateTime'] = raw_data['UpdateTime'].map(time_to_num)

In [6]:
raw_data['label'] = get_label(raw_data,10)

In [7]:
#make sure the label and feature are in the same day
data_with_label = raw_data[raw_data['UpdateTime'].shift(-10) >= raw_data['UpdateTime']]

In [8]:
data_with_label = data_with_label.dropna()
data_with_label = data_with_label.reset_index(drop=True)
data_with_label

,indicator1,indicator2,indicator3,indicator4,indicator5,indicator6,indicator7,indicator8,indicator9,indicator10,...,AskVolume1,BidVolume1,BidVolume2,BidVolume3,BidVolume4,BidVolume5,OpenInterest,UpperLimitPrice,LowerLimitPrice,label
0,1.000090,-0.086505,-1374.480,-2.076120,0.000178,0.999911,0.998495,0.750000,1.038060,0.004270,...,2,1,28,4,62,2,89336,5869,5311,1.0
1,1.000090,-0.086505,-1374.480,0.000000,0.000178,0.999911,0.998506,0.500000,0.000000,0.000000,...,2,1,28,4,62,2,89336,5869,5311,1.0
2,1.000090,-0.086505,-1374.480,0.000000,0.000178,1.000090,0.998516,0.500000,0.000000,0.000712,...,2,1,28,4,62,2,89336,5869,5311,1.0
3,1.000090,0.000000,490.311,1.038060,0.000178,1.000090,0.998703,0.666667,0.346021,0.002134,...,2,2,1,28,7,62,89340,5869,5311,0.0
4,1.000090,0.000000,571.371,0.000000,0.000178,1.000090,0.998712,0.500000,0.000000,0.000000,...,2,2,1,28,7,62,89340,5869,5311,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1721122,1.000190,0.081699,-2152.940,-1.633990,0.000378,1.000000,0.999881,0.500000,0.000000,0.003771,...,7,8,23,24,16,12,326050,5443,4925,0.5
1721123,0.999716,0.429553,2063.490,1.202750,0.000567,0.999754,1.000450,0.650000,0.515464,0.003777,...,1,6,23,1,24,16,326056,5443,4925,1.5
1721124,0.999622,-8.676980,-47445.400,8.591070,0.000757,0.999996,1.000630,0.240000,-4.467350,0.018512,...,107,6,23,1,24,16,326004,5443,4925,1.0
1721125,0.999811,-8.591070,-46540.200,0.000000,0.000378,0.999996,1.000630,0.500000,0.000000,0.000000,...,107,7,6,23,2,24,326004,5443,4925,0.0


In [9]:
tmp = data_with_label['UpdateTime'].shift(-1) - data_with_label['UpdateTime']

In [10]:
end_day = np.where(tmp < 0)[0].tolist()

In [11]:
begin = 0
data_in_day = []
for i in end_day:
    data_in_day.append(data_with_label[begin: i + 1])
    begin = i + 1
data_in_day.append(data_with_label[begin: data_with_label.shape[0]])

In [12]:
trainning_set, testing_set = train_test_split(data_in_day, test_size = 0.2, random_state = 37)

In [13]:
len(trainning_set)

36

In [14]:
len(testing_set)

9

In [15]:
training_index = [i for d in trainning_set for i in d.index]
testing_index = [i for d in testing_set for i in d.index]
print(len(training_index), training_index[:10], '...', training_index[-10:])
print(len(testing_index), testing_index[:10], '...', testing_index[-10:])
np.save('data/many_train.npy', training_index)
np.save('data/many_test.npy', testing_index)

1387896 [25381, 25382, 25383, 25384, 25385, 25386, 25387, 25388, 25389, 25390] ... [623931, 623932, 623933, 623934, 623935, 623936, 623937, 623938, 623939, 623940]
333231 [503491, 503492, 503493, 503494, 503495, 503496, 503497, 503498, 503499, 503500] ... [1624082, 1624083, 1624084, 1624085, 1624086, 1624087, 1624088, 1624089, 1624090, 1624091]
